In [0]:
# Importing Libraries

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir('/content/drive/My Drive/AppliedAI/HAR')

In [0]:
!pip install hyperas

In [0]:
import pandas as pd
import numpy as np
import hyperas as hp
import hyperopt
from hyperas import optim
from hyperas.distributions import choice
from hyperopt import Trials, STATUS_OK, tpe

Using TensorFlow backend.


In [0]:
import pandas as pd
import numpy as np
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import SeparableConv2D, MaxPooling2D, BatchNormalization, Conv2D
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [0]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    #X_train, X_test = load_signals('train'), load_signals('test')
    #y_train, y_test = load_y('train'), load_y('test')
    
    SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
    ]
    
    subset = 'train'
    signals_data = []
    
    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            pd.read_csv(filename, delim_whitespace=True, header=None).as_matrix()
        )
    X_train = np.transpose(signals_data, (1, 2, 0))
    
    ##########################
    
    subset = 'test'
    signals_data = []
    
    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            pd.read_csv(filename, delim_whitespace=True, header=None).as_matrix()
        )
    X_test = np.transpose(signals_data, (1, 2, 0))
    
    ###########################
    
    subset = 'train'
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = pd.read_csv(filename, delim_whitespace=True, header=None)[0]
    Y_train = pd.get_dummies(y).as_matrix()
    
    ############################
    
    subset = 'test'
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = pd.read_csv(filename, delim_whitespace=True, header=None)[0]
    Y_test = pd.get_dummies(y).as_matrix()
    
    

    return X_train, Y_train, X_test, Y_test

In [0]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [0]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM, Activation, Input, BatchNormalization
from keras.layers.core import Dense, Dropout

In [0]:
# Initializing parameters
epochs = 20
batch_size = 64
#n_hidden = 32

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [0]:
# Loading the train and test data
X_train, Y_train, X_test, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [0]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [0]:
X_train.shape

(7352, 128, 9)

- Defining the Architecture of LSTM

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='softmax'))
model.summary()

In [0]:
import keras

In [0]:
#@title
K.clear_session()

input = Input(shape=(timesteps, input_dim))

lstm1 = LSTM(128, return_sequences=True, kernel_initializer='glorot_normal')(input)
residue1 = keras.activations.linear(lstm1)

lstm2 = LSTM(128, return_sequences=True, kernel_initializer='glorot_normal')(lstm1)
drop2 = Dropout(0.5)(lstm2)

res_layer = keras.layers.Add()([residue1, drop2])
lstm3 = LSTM(64, return_sequences=True, kernel_initializer='glorot_normal')(res_layer)
drop3 = Dropout(0.5)(lstm3)

#####

lstm4 = LSTM(64, return_sequences=True, kernel_initializer='glorot_normal')(lstm3)
residue2 = keras.activations.linear(lstm4)

lstm5 = LSTM(64, return_sequences=True, kernel_initializer='glorot_normal')(lstm4)
drop5 = Dropout(0.5)(lstm5)

res_layer2 = keras.layers.Add()([residue2, drop5])
lstm6 = LSTM(128, return_sequences=True, kernel_initializer='glorot_normal')(res_layer2)
drop6 = Dropout(0.5)(lstm6)

#####

lstm7 = LSTM(64, return_sequences=True, kernel_initializer='glorot_normal')(drop6)
residue3 = keras.activations.linear(lstm7)

lstm8 = LSTM(64, return_sequences=True, kernel_initializer='glorot_normal')(lstm7)
drop8 = Dropout(0.5)(lstm8)

res_layer = keras.layers.Add()([residue3, drop8])
lstm9 = LSTM(128, kernel_initializer='glorot_normal')(res_layer)

dense1 = Dense(512, activation='relu', kernel_initializer='glorot_normal')(lstm9) 
drop9 = Dropout(0.5)(dense1)

dense2 = Dense(256, activation='relu', kernel_initializer='glorot_normal')(drop9)
drop10 = Dropout(0.4)(dense2)

output = Dense(n_classes, activation='softmax')(drop9)

model = keras.models.Model([input], [output])
model.summary()




In [0]:
#@title
# Compiling the model
opt = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [0]:
#@title
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=512,
          validation_data=(X_test, Y_test),
          epochs=60)

In [0]:
#@title
#################### NEW MODEL1 ######################
#K.clear_session()

def model(X_train, Y_train, X_test, Y_test):
    model = Sequential()

    model.add(LSTM({{choice([32, 64, 128])}}, input_shape=(128, 9), return_sequences=True, kernel_initializer='glorot_normal'))
    model.add(Dropout(0.5))

    #model.add(keras.layers.Reshape((128, 1)))
    model.add(LSTM({{choice([64, 128])}}, kernel_initializer='glorot_normal'))
    model.add(Dropout({{choice([0.3, 0.5])}}))


    model.add(Dense({{choice([128, 256, 512])}}, activation={{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout(0.5))

    '''if({{choice([1, 2])}}==2):
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))'''
        
        
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(6, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer={{choice(['rmsprop', 'adam'])}},
                  metrics=['accuracy'])
    
    model.fit(X_train, Y_train,
              batch_size={{choice([128, 256])}},
              epochs=1,
              verbose=1,
              validation_data=(X_test, Y_test))
    
    score, acc = model.evaluate(X_test, Y_test, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [0]:
#@title
best_run, best_model = optim.minimize(model=model,
                                          data=load_data,
                                          max_evals=25,
                                          algo=tpe.suggest,
                                          notebook_name='HAR_LSTM', 
                                          trials=Trials())

In [0]:
best_run

# BLOG MODEL :(ASSIGNMENT)

In [0]:
from sklearn.tree import DecisionTreeClassifier as DT
labels=['LAYING', 'SITTING','STANDING','WALKING','WALKING_DOWNSTAIRS','WALKING_UPSTAIRS']

train = pd.read_csv('/content/drive/My Drive/AppliedAI/HAR/UCI_HAR_Dataset/csv_files/train.csv')
test = pd.read_csv('/content/drive/My Drive/AppliedAI/HAR/UCI_HAR_Dataset/csv_files/test.csv')
print(train.shape, test.shape)

(7352, 564) (2947, 564)


In [0]:
#static 1 dynamic 0
binary_tr = [1 if int(x) in [4, 5, 6] else 0 for x in train['Activity'].values]
binary_test = [1 if int(x) in [4, 5, 6] else 0 for x in test['Activity'].values]
train['static'] = binary_tr
test['static'] = binary_test

train.head()
test.head()

,tBodyAccmeanX,tBodyAccmeanY,tBodyAccmeanZ,tBodyAccstdX,tBodyAccstdY,tBodyAccstdZ,tBodyAccmadX,tBodyAccmadY,tBodyAccmadZ,tBodyAccmaxX,tBodyAccmaxY,tBodyAccmaxZ,tBodyAccminX,tBodyAccminY,tBodyAccminZ,tBodyAccsma,tBodyAccenergyX,tBodyAccenergyY,tBodyAccenergyZ,tBodyAcciqrX,tBodyAcciqrY,tBodyAcciqrZ,tBodyAccentropyX,tBodyAccentropyY,tBodyAccentropyZ,tBodyAccarCoeffX1,tBodyAccarCoeffX2,tBodyAccarCoeffX3,tBodyAccarCoeffX4,tBodyAccarCoeffY1,tBodyAccarCoeffY2,tBodyAccarCoeffY3,tBodyAccarCoeffY4,tBodyAccarCoeffZ1,tBodyAccarCoeffZ2,tBodyAccarCoeffZ3,tBodyAccarCoeffZ4,tBodyAcccorrelationXY,tBodyAcccorrelationXZ,tBodyAcccorrelationYZ,...,fBodyBodyAccJerkMagmeanFreq,fBodyBodyAccJerkMagskewness,fBodyBodyAccJerkMagkurtosis,fBodyBodyGyroMagmean,fBodyBodyGyroMagstd,fBodyBodyGyroMagmad,fBodyBodyGyroMagmax,fBodyBodyGyroMagmin,fBodyBodyGyroMagsma,fBodyBodyGyroMagenergy,fBodyBodyGyroMagiqr,fBodyBodyGyroMagentropy,fBodyBodyGyroMagmaxInds,fBodyBodyGyroMagmeanFreq,fBodyBodyGyroMagskewness,fBodyBodyGyroMagkurtosis,fBodyBodyGyroJerkMagmean,fBodyBodyGyroJerkMagstd,fBodyBodyGyroJerkMagmad,fBodyBodyGyroJerkMagmax,fBodyBodyGyroJerkMagmin,fBodyBodyGyroJerkMagsma,fBodyBodyGyroJerkMagenergy,fBodyBodyGyroJerkMagiqr,fBodyBodyGyroJerkMagentropy,fBodyBodyGyroJerkMagmaxInds,fBodyBodyGyroJerkMagmeanFreq,fBodyBodyGyroJerkMagskewness,fBodyBodyGyroJerkMagkurtosis,angletBodyAccMeangravity,angletBodyAccJerkMeangravityMean,angletBodyGyroMeangravityMean,angletBodyGyroJerkMeangravityMean,angleXgravityMean,angleYgravityMean,angleZgravityMean,subject,Activity,ActivityName,static
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,-0.554577,-0.466223,0.717208,0.635502,0.789497,-0.877764,-0.997766,-0.998414,-0.934345,-0.975669,-0.949824,-0.830478,-0.168084,-0.378996,0.246217,0.521204,-0.487793,0.482280,-0.045462,0.211955,-0.134894,0.130858,-0.014176,-0.105971,0.073544,-0.171516,0.040063,0.076989,-0.490546,-0.709003,...,-0.035356,-0.254248,-0.700326,-0.770610,-0.797113,-0.764485,-0.820188,-0.937959,-0.770610,-0.970958,-0.798387,0.179435,-1.000000,-0.047391,-0.467849,-0.761326,-0.890165,-0.907308,-0.895301,-0.917883,-0.909829,-0.890165,-0.994105,-0.898022,-0.234815,-1.000000,0.071645,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978,2,5,STANDING,1
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,-0.554577,-0.806013,0.768031,0.683698,0.796706,-0.969097,-0.999580,-0.999646,-0.997720,-0.994006,-0.973637,-0.950720,-0.302437,-0.348243,-0.404785,0.507492,-0.156495,0.040674,0.272991,0.197566,-0.194556,0.411411,-0.340466,0.077555,-0.084024,0.035305,-0.010083,-0.104983,-0.429134,0.399177,...,-0.491213,-0.215404,-0.666004,-0.924461,-0.916774,-0.919383,-0.917983,-0.991121,-0.924461,-0.995727,-0.933127,-0.218860,-1.000000,-0.031474,-0.090655,-0.461801,-0.951977,-0.938212,-0.938008,-0.949922,-0.986902,-0.951977,-0.998272,-0.951255,-0.431053,-1.000000,-0.401189,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898,2,5,STANDING,1
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,-0.568512,-0.799116,0.848305,0.667864,0.822442,-0.976782,-0.999954,-0.999570,-0.998720,-0.993456,-0.974266,-0.964976,-0.617916,-0.695490,-0.536693,0.242147,-0.114997,0.032703,0.192385,-0.011945,-0.063363,0.470819,-0.507395,0.188536,-0.231575,0.632120,-0.550708,0.305653,-0.323848,0.279786,...,0.140729,-0.206062,-0.639809,-0.975209,-0.973998,-0.967097,-0.982849,-0.997663,-0.975209,-0.999504,-0.968260,-0.561266,-0.897436,-0.168805,-0.662317,-0.925364,-0.985689,-0.983273,-0.983422,-0.986857,-0.989609,-0.985689,-0.999831,-0.986148,-0.809976,-0.936508,0.062891,-0.190422,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346,2,5,STANDING,1
3,0.270298,-0.032614,-0.117520,-0.994743,-0.973268,-0.967091,-0.995274,-0.974471,-0.968897,-0.938610,-0.568512,-0.799116,0.848305,0.667864,0.822442,-0

In [0]:
from keras import backend as K
K.clear_session()

In [0]:
clf = DT(max_depth=4, min_samples_split=100, random_state=41)
y_dt = train['static']
clf.fit(train.drop(['static', 'Activity', 'ActivityName'], axis=1), y_dt)
from sklearn.metrics import roc_auc_score
roc_auc_score(test['static'], clf.predict(test.drop(['static', 'Activity', 'ActivityName'], axis=1)))

1.0

In [0]:
static_train = train[train['static']==1]
print(static_train.shape)
dynamic_train = train[train['static']==0]
print(dynamic_train.shape)


static_test = test[test['static']==1]
print(static_test.shape)
dynamic_test = test[test['static']==0]
print(dynamic_test.shape)

(4067, 565)
(3285, 565)
(1560, 565)
(1387, 565)


In [0]:
y_train_static = keras.utils.to_categorical(static_train['Activity']-min(static_train['Activity']))
y_train_dynamic = keras.utils.to_categorical(dynamic_train['Activity']-min(dynamic_train['Activity']))

In [0]:
y_test_static = keras.utils.to_categorical(static_test['Activity']-min(static_test['Activity']))
y_test_dynamic = keras.utils.to_categorical(dynamic_test['Activity']-min(dynamic_test['Activity']))

In [0]:
y_train_static.shape

(4067,)

In [0]:
from keras import backend as K
K.clear_session()

################# STATIC MODEL ##################
input_static = Input(shape=(561,))
static_reshape = keras.layers.Reshape((561, 1))(input_static)

conv1 = keras.layers.Conv1D(30, 3, activation='relu')(static_reshape)
conv2 = keras.layers.Conv1D(50, 3, activation='relu')(conv1)
conv3 = keras.layers.Conv1D(100, 3, activation='relu')(conv2)

flattened = keras.layers.Flatten()(conv3)

dense1 = Dense(512, activation='relu')(flattened)
drop = Dropout(0.5)(dense1)

output_static = Dense(3, activation='softmax')(drop)

static_model = Model([input_static], [output_static])
static_model.summary()

W0827 08:28:28.739610 140706282710912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0827 08:28:28.742276 140706282710912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0827 08:28:28.785339 140706282710912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0827 08:28:28.786756 140706282710912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W082

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 561)               0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 561, 1)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 559, 30)           120       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 557, 50)           4550      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 555, 100)          15100     
_________________________________________________________________
flatten_1 (Flatten)          (None, 55500)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               28416512  
__________

In [0]:
################## DYNAMIC MODEL ####################

input_dynamic = Input(shape=(561,))
dynamic_reshape = keras.layers.Reshape((561, 1))(input_dynamic)
conv1_dy = keras.layers.Conv1D(100, 3)(dynamic_reshape)
max_pool = keras.layers.MaxPooling1D(3, 3)(conv1_dy)

flattened_dy = keras.layers.Flatten()(max_pool)

dense1_dy = Dense(512, activation='relu')(flattened_dy)
drop_dy = Dropout(0.5)(dense1_dy)

output_dynamic = Dense(3, activation='softmax')(drop_dy)

dynamic_model = Model([input_dynamic], [output_dynamic])


W0827 08:28:49.575448 140706282710912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [0]:
############ TRAINING DYNAMIC MODEL ###################

dynamic_model.compile(loss='categorical_crossentropy',
                    optimizer=keras.optimizers.Adam(lr=0.0005),
                    metrics=['accuracy'])

dynamic_model.fit( dynamic_train.drop(['Activity', 'subject', 'ActivityName', 'static'], axis=1), y_train_dynamic,
                epochs=50,
                verbose=1,
                batch_size=128,
                validation_data=(dynamic_test.drop(['Activity', 'subject', 'ActivityName', 'static'], axis=1), y_test_dynamic))

W0827 08:28:55.628895 140706282710912 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0827 08:28:55.923820 140706282710912 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 3285 samples, validate on 1387 samples
Epoch 1/50
3285/3285 [==============================] - 12s 4ms/step - loss: 0.6341 - acc: 0.7394 - val_loss: 0.3635 - val_acc: 0.8709
Epoch 2/50
3285/3285 [==============================] - 11s 3ms/step - loss: 0.2043 - acc: 0.9419 - val_loss: 0.1977 - val_acc: 0.9394
Epoch 3/50
3285/3285 [==============================] - 11s 3ms/step - loss: 0.1190 - acc: 0.9689 - val_loss: 0.2051 - val_acc: 0.9286
Epoch 4/50
3285/3285 [==============================] - 11s 3ms/step - loss: 0.0816 - acc: 0.9778 - val_loss: 0.1928 - val_acc: 0.9315
Epoch 5/50
3285/3285 [==============================] - 11s 3ms/step - loss: 0.0572 - acc: 0.9878 - val_loss: 0.1462 - val_acc: 0.9488
Epoch 6/50
3285/3285 [==============================] - 11s 3ms/step - loss: 0.0484 - acc: 0.9875 - val_loss: 0.1220 - val_acc: 0.9567
Epoch 7/50
3285/3285 [==============================] - 11s 3ms/step - loss: 0.0356 - acc: 0.9909 - val_loss: 0.1285 - val_acc: 0.9553
Epoch 8

In [0]:
############ TRAINING STATIC MODEL ###################

static_model.compile(loss='categorical_crossentropy',
                    optimizer=keras.optimizers.Adam(lr=0.0001),
                    metrics=['accuracy'])

static_model.fit( static_train.drop(['Activity', 'subject', 'ActivityName', 'static'], axis=1), y_train_static,
                epochs=100,
                verbose=1,
                batch_size=128,
                validation_data=(static_test.drop(['Activity', 'subject', 'ActivityName', 'static'], axis=1), y_test_static))

Train on 4067 samples, validate on 1560 samples
Epoch 1/100
4067/4067 [==============================] - 47s 11ms/step - loss: 0.8708 - acc: 0.6467 - val_loss: 0.5768 - val_acc: 0.8853
Epoch 2/100
4067/4067 [==============================] - 45s 11ms/step - loss: 0.4338 - acc: 0.8589 - val_loss: 0.3060 - val_acc: 0.8949
Epoch 3/100
4067/4067 [==============================] - 45s 11ms/step - loss: 0.2760 - acc: 0.8901 - val_loss: 0.2310 - val_acc: 0.9269
Epoch 4/100
4067/4067 [==============================] - 45s 11ms/step - loss: 0.2028 - acc: 0.9233 - val_loss: 0.1773 - val_acc: 0.9321
Epoch 5/100
4067/4067 [==============================] - 45s 11ms/step - loss: 0.1601 - acc: 0.9407 - val_loss: 0.1432 - val_acc: 0.9519
Epoch 6/100
4067/4067 [==============================] - 45s 11ms/step - loss: 0.1305 - acc: 0.9518 - val_loss: 0.1257 - val_acc: 0.9615
Epoch 7/100
4067/4067 [==============================] - 46s 11ms/step - loss: 0.1121 - acc: 0.9584 - val_loss: 0.1286 - val_acc: 

In [0]:
from sklearn.metrics import accuracy_score
def Eval(X_test, y_test, DT, static_model, dynamic_model, y_actual_static_test, y_actual_dynamic_test):
    
    total_points = X_test.shape[0]
    X_test.index = range(total_points)
    y_pred = DT.predict(X_test.drop(['Activity'], axis=1))
    #print(type(y_pred))
    static, dynamic = 0, 0
    
    
    for i, val in enumerate(y_pred):
        
        if(val == 0):
            dynamic = dynamic + 1
            if(X_test['Activity'][i] not in [1, 2, 3]): #actually static
                X_test.drop(index=i)
                dynamic = dynamic - 1
                static = static +1
                y_actual_static_test = np.delete(y_actual_static_test, static, 0)
                y_pred = np.delete(y_pred, i, 0)
                 
        elif(val == 1):
            static = static + 1
            if(X_test['Activity'][i] not in [4, 5, 6]): #actually dynamic
                X_test.drop(index=i)
                static = static - 1
                dynamic = dynamic + 1
                y_actual_dynamic_test = np.delete(y_actual_dynamic_test, dynamic, 0)
                y_pred = np.delete(y_pred, i, 0)
               
    X_static = X_test.iloc[[bool(val) for val in y_pred]]
    X_dynamic = X_test.iloc[[not val for val in y_pred]]
    
    y_pred_static = static_model.predict(X_static.drop(columns=['Activity', 'subject']))
    y_pred_static = keras.utils.to_categorical(np.argmax(y_pred_static, axis=1))
    
    y_pred_dynamic = dynamic_model.predict(X_dynamic.drop(columns=['Activity', 'subject']))
    y_pred_dynamic = keras.utils.to_categorical(np.argmax(y_pred_dynamic, axis=1))
    
    '''print(y_actual_static_test)
    print('###############')
    print(keras.utils.to_categorical(np.argmax(y_pred_static, axis=1)))'''
    
    static_correct = accuracy_score(y_actual_static_test, y_pred_static, normalize=False)
    dynamic_correct = accuracy_score(y_actual_dynamic_test, y_pred_dynamic, normalize=False)
    
    score = (static_correct + dynamic_correct)/total_points
    
    return score
    

In [0]:
################# TEST PHASE ########################

score = Eval(test.drop(['ActivityName', 'static'], axis=1), test['Activity'], clf, static_model,\
             dynamic_model, y_test_static, y_test_dynamic) 

In [0]:
score

0.9660671869697998